# Load Data from apple_health_export/ 
Originally as XML file, want to convert to csv

In [3]:
# Load needed packages
import xml.etree.ElementTree as ET
import pandas as pd
import xmltodict

In [4]:
# use your own directory, this is where I exported my data
data_path = 'apple_health_export/export.xml' 

# read XML file, convert to a dictionary
with open(data_path, 'r') as xml_file:
    all_data = xmltodict.parse(xml_file.read())

In [6]:
# look at all data available (remember it is in a dictionary)
all_data['HealthData'].keys()

odict_keys(['@locale', 'ExportDate', 'Me', 'Record', 'Workout', 'ActivitySummary'])

For now, let's just look at my individual activity summary for each day and focus on that. Will put all health Records and Activity Summary into list, and then pandas dataframe

In [8]:
# create a records list for overall health data, put into pandas dataframe
records_list = all_data['HealthData']['Record']
all_records = pd.DataFrame(records_list)

In [ ]:
# activity summary (as)
as_list = all_data['HealthData']['ActivitySummary']
as_data = pd.DataFrame(as_list())

# Now that data is imported, I will clean it for easier use 

In [ ]:
## Cleaning 

# get datetime format where needed
for col in ['creationDate', 'startDate', 'endDate']:
    data[col] = pd.to_datetime(data[col])

# value is numeric, NaN if fails
data['value'] = pd.to_numeric(data['value'], errors='coerce')

# some records do not measure anything, just count occurences
# filling with 1.0 (= one time) makes it easier to aggregate
data['value'] = data['value'].fillna(1.0)

# shorter observation names: use vectorized replace function
data['type'] = data['type'].str.replace('HKQuantityTypeIdentifier', '')
data['type'] = data['type'].str.replace('HKCategoryTypeIdentifier', '')